In [1]:
import sys
sys.path.append('..')
import ee
ee.Initialize()
import geedatasets as dset

In [2]:
import geetools

In [3]:
import ipygee as ui

In [4]:
s1 = dset.fromShortName('S1')

In [5]:
print(s1)

ID: COPERNICUS/S1_GRD
Start Date: 2014-10-03
End Date: None
Bands: ['HH (HH)', 'HV (HV)', 'VH (VH)', 'VV (VV)', 'angle (angle)']
Visualizers: None



In [6]:
Map = ui.Map()
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [7]:
p = ee.Geometry.Point([-72,-42])
site = p.buffer(1000)

In [8]:
Map.addLayer(ee.Feature(site), name='site')
Map.centerObject(site)

In [9]:
col = s1.collection().filterBounds(site).filterDate('2020-01-01', '2020-02-01')

In [10]:
vv_vh = s1.filter_polarisation('VV-VH', col)

In [11]:
# Now that you get an homogeneous collection you can create a composite
vv_vh_median = vv_vh.median()
vv_vh_median = vv_vh_median.addBands(vv_vh_median.select('VH').subtract(vv_vh_median.select('VV')).rename('diff'))
params = geetools.visualization.stretch_percentile(vv_vh_median, site, ['VV', 'VH', 'diff'], scale=10)
Map.addLayer(vv_vh_median, params, name='VV Median')

In [12]:
# LEE Speckle Filter
vv_vh_lee = vv_vh.map(lambda i: s1.lee_filter(i))
vv_vh_median_lee = vv_vh_lee.median()
vv_vh_median_lee = vv_vh_median_lee.addBands(vv_vh_median_lee.select('VH').subtract(vv_vh_median_lee.select('VV')).rename('diff'))
params_lee = geetools.visualization.stretch_percentile(vv_vh_median_lee, site, ['VV', 'VH', 'diff'], scale=10)
Map.addLayer(vv_vh_median_lee, params, name='VV-VH Median (Lee Filter)')

In [13]:
# RADAR VEGETATION INDEX

In [14]:
rvi = vv_vh_lee.map(lambda i: s1.RVI.apply(i))

In [15]:
rvi_median = rvi.median()

In [17]:
Map.addLayer(rvi_median, dict(min=0, max=1), 'median RVI')